In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423


In [2]:
using CUDA
CUDA.device!(1)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


CuDevice(1): TITAN RTX

In [3]:
q=21
ctype=Symbol("amino")
typel=Symbol("bm")
T = Float32
muext = 0.50;
muint = 2.50;

In [4]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

ParamAlgo{Float32}
-------------
damp=0.0
tol=1.0e-5
tolnorm=1.0e-5
tmax=10
upscheme=sequential
lr=sce
beta=1.0
verbose=true
-------------

In [5]:
(N, L) = (6, 5) 

using Random
header = "myseq"
myseq = randstring('A':'Z', N)
seq = Seq(header, myseq, ctype)

lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [6]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=5 N=6 q=21 size=43.547 KiB]

In [7]:
bpm = BPMessages(seq, pm)

BPMessages{Float32}[L=5 N=6 ongpu=true size=26.914 KiB]

In [8]:
bpb = BPBeliefs(N, L)

BPBeliefs{Float32}[L=5 N=6 ongpu=true size=30.625 KiB]

In [9]:
lrf = LongRangeFields(N, L)

LongRangeFields{Float32}[L=5 N=6 ongpu=true size=105.312 KiB]

In [10]:
af = AllFields(bpm, bpb, lrf)

AllFields{Float32}[L=5 N=6 ongpu=true size=162.852 KiB]

In [11]:
BpAlignGpu.update_F!(af, pm, pa)
BpAlignGpu.update_hF!(af, pm, pa)
BpAlignGpu.update_B!(af, pm, pa)
BpAlignGpu.update_hB!(af, pm, pa)
BpAlignGpu.update_beliefs!(af, pm, pa)
BpAlignGpu.update_jointchain!(af, pm, pa)
BpAlignGpu.update_conditional_chain!(af, pa)
BpAlignGpu.update_conditional_all!(af, pm)


In [12]:
BpAlignGpu.update_f!(af)
BpAlignGpu.update_g!(af)


In [13]:
BpAlignGpu.lr_freeen(af, pm)

5.418040037155151

In [14]:
BpAlignGpu.logZi(af, pm, pa)

-17.616098f0

In [15]:
BpAlignGpu.logZa(af, pm, pa)

15.527098f0

In [16]:
BpAlignGpu.logZia(af, pm)

-19.445797f0

In [17]:
resfm = BpAlignGpu.update_f_memory!(af);

In [18]:
af.lrf.f == resfm

true

In [19]:
resgm = BpAlignGpu.update_g_memory!(af);

In [20]:
af.lrf.g == resgm

true

In [21]:
BpAlignGpu.one_bp_sweep!(af, pm, pa)

In [22]:
BpAlignGpu.test_sweep!(100,af,pm,pa)

t=1	 err=0.16357356
t=2	 err=0.13758701
t=3	 err=0.033801064
t=4	 err=0.019929945
t=5	 err=0.011904612
t=6	 err=0.005274996
t=7	 err=0.003157951
t=8	 err=0.0016046911
t=9	 err=0.0006352663
t=10	 err=0.000502795
t=11	 err=0.0002681762
t=12	 err=0.00011597574
t=13	 err=5.441904e-5
t=14	 err=4.351139e-5
t=15	 err=2.3514032e-5
t=16	 err=8.0913305e-6
converged: err=8.0913305e-6, tol=1.0e-5
